# <span style="color:#f37726">03 RKI Model Selection</span>

**Ziele des Notebooks:**
- Vorhersagemodelle motivieren
- Kriterien der Fallzahlenprognose definieren
- Geeignete Modelle vorstellen
- Vor- und Nachteile erötern
- Implementierbarkeit untersuchen
- Feature Engineering skizieren
- Evaluierungskriterien festsetzen

## Motivation

Zeitliche Vorhersagemodelle prognostizieren den Wert einer abhängigen Variable mithilfe eines funktionalen Zusammenhanges sowie Daten über die Entwicklung ebendieser Variable in der Vergangenheit. Dabei müssen nicht zwangsläufig Informationen über die erklärende/n, also unabhängige/n, Variable/n vorliegen. Einen solchen Ansatz verfolgen beispielsweise Naïve Modelle, Trend-Saison-Modelle, exponentielle Glättungsmodelle, Autoregressive Modelle und verschiedene Netzstrukturen (LSTM, Autoencoder, CNN, RNN). Naïve Modelle werden dabei oft als Benchmark verwendet. Alternativ lassen sich Vorhersagen auch über unabhängige Schätzvariablen treffen. Solche Frühindikatoren müssen mit der abhängigen Variable korrelieren. Ein kausaler Zusammenhang ist jedoch nicht notwendig. Verschiedene Regressionsmodelle, mehrlagige Perzeptren, Support Vector Machines und Autoregressive Modelle mit exogenen Inputs (ARMAX) basieren auf diesem Ansatz.

## Modellkriterien

Jedes der genannten Modelle zeichnet sich durch individuelle Eigenschaften mit Stärken und Schwächen, im Bezug zum Anwendungsfall, aus. Die Wahl des Vorhersagemodells muss daher unbedingt an die Struktur der vorherzusagenden Variable, den Datensatz, den Vorhersagezeitraum und die Evaluierungskriterien angepasst werden.

**Struktur des Datensatzes**

Durch die parallele Erfassung der Fallzahlen in den 401 Landkreisen handelt es sich bei dem vom Robert Koch-Institut veröffentlichten Datensatz um Paneldaten. Paneldaten haben neben einer zeitlichen auch eine objektspezifische Dimension. Durch die Selektion eines Objektes erhält man dessen Zeitreihendaten. Da die meisten Vorhersagemodelle lediglich Zeitreihen für ein Objekt verwerten können, ist ggf. eine Dimensionsreduktion  notwendig.

**Abhängige Variable (Fallzahlen)**

Für eine optimale Anpassung des Vorhersagemodells an den globalen Funktionsgraphen werden Informationen über dessen Eigenschaften benötigt:

*1) Stationarität* <br>
<br>Die Zeitreihe einer abhänigen Variable gilt als stationär, wenn sich deren Wahrscheinlichkeitsverteilung im Zeitverlauf nicht verändert. Die Entwicklung der Fallzahlen in Deutschland weißt langfristig keine Stationarität auf. Die entsprechende Wahrscheinlichkeitsverteilung hängt von dynamischen Faktoren wie den Testkapazitäten, der Impfqoute, der Bevölkerungsmobilität und den Kontaktbeschränkungen ab. Deutlich wird dies, wenn man die Fallzahlen Liniendiagramm darstellt:

![](visualization/bund_time_cases_lineplot.svg)

*2) Trend*

Ein Trend beschreibt eine langfristige und nachhaltige Veränderung einer Zeitreihenvariable in eine Richtung. Er ist unabhängig von kurzfristigen und allgemeinen Schwankungen. Ein Beispiel ist das jährliche Wachstum des BIPs in Industrieländern (~2% pro Jahr). Über den gesamten Zeitraum der Pandemie weisen die COVID-19-Fallzahlen keinen Trend auf. Ob es unterjährig, also über einen kurzen Zeitraum, trendartige Entwicklungen (z.B. exponentielles Wachstum im letzten Quartal 2020) gegeben hat ist strittig. Für das Vorhersagemodell wird angenommen, dass kein Trend vorliegt.

*3) Strukturbruch*

Strukturbrüche entstehen, wenn sich die Regressionsparameter im Zeitverlauf signifikant ändern. Versucht man beispielsweise den Funktionsgraph in der oberen Abbildung mit drei bis vier linearen, lokalen Regressionsgeraden anzunähern, dann würde man jeweils signifikant verschiedene Steigungsparameter erhalten (einen neutralen/leicht Positiven, einen stark Positiven und einen moderat Negativen). Die Entwicklung der Fallzahlen in Deutschland weist also mehrere Strukturbrüche auf. Dessen Ursprung könnten zum Beispiel angeordnete Massenquarantänemaßnahmen (Lockdowns) sein.

*4) Saisonalität*

Saisonalität beschreibt eine wiederkehrende Variation der Merkmalsausprägung innerhalb spezifischer zeitlicher Intervalle (jährlich, wöchentlich, stündlich, etc.). Saisoneffekte können in einer Zeitreihe auch parallel auftreten (z.B. erhöhtes online Bestellvorkommen in den Abendstunden und vor Weihnachten). Durch die bereits angesprochene Meldekette weist die Entwicklung der Fallzahlen einen starken, wöchentlichen Saisoneffekt auf. Fraglich bleibt, ob eventuell auch jährlich ein Saisoneffekt vorliegt, wenn zum Beispiel die Fallzahlen immer in den Sommermonaten sinken (ähnlich wie bei der durch Influenca-Viren ausgelösten Grippe). Aufgrund fehlender Daten kann dazu zum jetzigen Zeitpunkt noch keine Aussage getroffen werden. Saisoneffekte lassen sich durch Glättung bereinigen.

**Vorhersagezeitraum**

Im Kontext der Fallzahlenentwicklung können Vorhersagen für einen Tag, eine Woche, ein Jahr oder gar bis zu einer eventuellen Ausrottung der Krankheit getroffen werden. Bei dieser arbiträren Wahl sollte jedoch beachtet werden, dass die Prognosegüte mit der Länge des Vorhersagezeitraumes abnimmt. Vorhersagemodelle werden zudem in Single- und Mutli-Period-Forecasts eingeteilt. Erstere treffen Vorhersagen für mehrere Zeitpunkte durch iterative Wiederholung der Prognose. Dabei wird der Vorhersagewert des letzten Zeitpunktes als Tatsache für die nächste Vorhersage angenommen. Mutli-Period-Forecasts hingegen geben direkt die Vorhersagen für alle Zeitpunkte aus. Als Zeitraum für die Prognose haben wir eine Woche gewählt. Wir empfinden dies als guten Trade-off zwischen Prognosegüte und Nutzen der Vorhersage.

**Vorhersageraum**

Die Vorhersagen sollen auf Landkreisebene getroffen werden. Im Fall Neuronaler Netze (oder anderer Modelle die trainiert werden müssten) stehen zwei Implementierungsmöglichkeiten zur Wahl. Entweder trainiert man ein globales Modell und wendet es dann lokal an oder man trainiert pro Landkreis ein eigenes Modell. In Ermangelung ausreichender Trainingsdaten und Rechenleistung wird der erste Ansatz gewählt.

**Exogene Indikatorvariable**

Ein mit den Fallzahlen korreliertes Attribut, welches einen vorlaufenden Charakter hat, könnte als Vorhersagegrundlage verwendet werden. Dabei muss das Attribut nicht die Ursache für Veränderungen in der Fallzahlenentwicklung sein, sondern lediglich möglichst pro- oder antizyklisch vorlaufen. Analoges Beispiel ist die Vorhersage von Regen anhand der Anzahl von Regenschirmen, welche Personen auf der Straße bei sich tragen. Folgende Beobachtungen könnten die Fallzahlenentwicklung indizieren:

- Bevölkerungsmobilität / Anzahl der Kontakte pro Person (kausal)
- Verbreitung hygienischer Präventationsmaßnahmen (kausal)
- Verkaufszahlen spezieller Medikamente oder Selbsttests (nicht kausal & unklar ob vorlaufend)
- Auslastungsquote der Krankenhäuser (nicht kausal & unklar ob vorlaufend)

Erwähnte Phänomene qualitativ hochwertig zu messen ist herausfordernd. Entsprechende, für die Öffentlichkeit zugängliche, Datensätze sind uns nicht bekannt.

## Modellauswahl

Die Eigenschaften der Zeitreihe schränken die Menge potenzieller Vorhersagemodelle ein. 
<br>Hier die Anforderungen, die unser Vorhersagemodell erfüllen **muss**:

- Vorhersage für sieben Tage (iterative Single-Period- oder Multi-Period-Vorhersage)
- Berücksichtigung der Nicht-Stationarität
- Robustheit trotz Strukturbrüchen
- Modellierung der wöchentlichen Saisonalität
- Anwendbarkeit auf Landkreisebene

Weitere Faktoren die **optional** berücksichtigt werden können:

- Impffortschritt / infektionsgefährdete Bevölkerung
- Struktur der Landkreise
- Geographische Lage

**1) Naïve / SNaïve**

Naïve Modelle basieren ihre Vorhersage exklusiv auf den zuletzt beobachteten Datenpunkten. Im Kontext der Fallzahlenentwicklung entspricht die Vorhersage der Fallzahlen zum Tag t den Fallzahlen am Tag t-1. Da die Saisonkomponente innerhalb einer Woche sehr dominant ist, soll statt dem Vortag der Wert vor einer Woche verwendet werden. Man spricht von einem S(easonal)Naïve-Ansatz. Formal dargestellt:

\begin{equation}
\widetilde{Y_t} = Y_{t-7}
\end{equation}

**2) SNaïve mit wöchentlicher Trendkomponente**

Als Erweiterung soll ein SNaïve-Modell mit kurzfristiger Trendkomponente verwendet werden. Die Trendkomponente entspricht dem Wachstum von Vortag zur Vorwoche:

\begin{equation}
\widetilde{Y_t} = Y_{t-7} + Y_{t-7}* \frac{Y_{t-1}}{Y_{t-8}}
\end{equation}

\begin{equation}
\widehat{=} \quad \widetilde{Y_t} = Y_{t-7}* \frac{Y_{t-1}}{Y_{t-8}}
\end{equation}

**3) Exponentielle Glättung**

Bei der exponentiellen Glättung wird der Prognosefehler der Vorperiode miteinbezogen. Der Glättungsfakor Alpha stellt dabei dessen Gewichtung dar. Werte zwischen 0 (Naïve-Ansatz) und 1 (keine Glättung) können gewählt werden:

\begin{equation}
\widetilde{Y_t}= Y_{t-7} + \alpha * (Y_{t-7} - \widetilde{Y}_{t-7})
\end{equation}

**4) ARIMA**

Autoregressive-moving-average (ARMA) Vorhersagemodelle prognostizieren mithilfe von Linearkombinationen vergangener Merkmalsausprägungen und gleitender Mittel entsprechender Prognosefehler. Dieser klassische Ansatz setzt Stationarität voraus. Bei nicht-stationären Zeitreihen (wie der Fallzahlenentwicklung) kann diese künstlich durch Differenzbildung induziert werden. Man spricht dann von Autoregressiven-integrated-moving-average (ARIMA) Modellen.

Die Parameter von ARIMA-Modellen werden meist in der Form ARIMA(p, d, q) (P, D, Q) (s) angegeben.

**5 & 6) MLP**

Neuronale Netze wandeln einen Eingabevektor in einen Ausgabevektor um. Als simples Modell für den Einstieg soll ein Mehrlagiges Perzeptron mit vollständig verbundenen Schichten verwendet werden.

Für eine Erweiterung des Eingabevektor kommen folgende Attribute in Betracht:
- Fallzahlen benachbarter Landkreise
- Weitere geographische Informationen (Bundesland, Grenzt an Ausland, etc.)
- Zeitliche Informationen (Wochentag, Monat)
- Struktur des Landkreises / Ergebnisse des Clusterings
- Impffortschritt bzw. die infektionsgefährdete Bevölkerung

Die Netze können unabhängig vom eigentlichen Datum trainiert werden. Somit werden die Paneldaten um die zeitliche Dimension reduziert. Stattdessen sollen die Fallzahlen der letzten 7 Tage als (Haupt-)Features verwendet werden.

**7) RNN (LSTM)**

Rekurrente Neuronale Netze unterscheiden sich zu Feedforward-Netzen durch Neuronenverbindungen innerhalb bzw. zu einer vorgelagerten Schicht. Sie bieten sich für Zeitreihendaten an, da die bereits getätigte Prognosen für neue Vorhersagen berücksichtigt werden. Wesentlicher Perfomanzfortschritt war die Implementierung eines Long short-term memory. Dabei werden in einer LSTM-Zelle Trainingserfahrungen abgespeichert. Wiederholen sich ebendiese, prägt sich ein Langzeitgedächntnis aus. Einmalige Erfahrungen hingegen werden nach kurzer Zeit wieder verworfen. Rekurrente Neuronale Netze mit Long short-term memory entfalten auf großen Datensätzen ihr Potenzial am besten.

Wesentlicher Aspekt Rekurrent Neuronaler Netze ist die Berücksichtigung der zeitlichen Datenstruktur. Während der Trainingsphase wird geordnet über die Trainingsdaten itteriert. Dabei ist die Eingabe ein Tensor mit den Dimensionen Batch / Timestamp / Features. Je Batch wird ein Vektor in der Form Timestamp / Zielattribut ausgegeben. Dieser ist Grundlage für die Bestimmung des Trainingsfehlers und somit für die Gewichtsanpassung.

##  Evaluierung

Die eben erwähnten Modelle sind nach ihrer Komplexität aufsteigend sortiert. Modelle sind vereinfachte Abbilder der Wirklichkeit. Die Entwicklung der COVID-19-Fallzahlen ist ein komplexes Phänomen mit vielen Einflussfaktoren. Intuitiv würde man zur Beschreibung ein möglichst komplexes Modell wählen. Das dies nicht immer erstebenswert ist, wurde in der Maxime von Ockham festgehalten ('Einfache Modelle sind besser'). Auch beim Trade-Off von Verzerrung und Varianz wird dies deutlich.

Für das Benchmarking der Modelle soll die Prognose mit den tatsächlichen Fallzahlen in vier Landkreisen verglichen werden. Um eine möglichst große geographische Diversität zu erreichen, haben wir uns für folgende Landkreise entschieden:
- Landkreis Anhalt-Bitterfeld (Standort der Hochschule)
- Kreisfreie Stadt Hamburg (Metropole)
- Landkreis Ortenau (ländlicher, westdeutscher Landkreis mit Grenze zu Frankreich)
- Sächsische Schweiz-Osterzgebirge (Kreis mit einen der höchsten Inzidenzwerten)

Während die Aufteilung in Trainings- und Testdaten bei maschinellen Lernmodellen essentiell ist, verzichten statistische Vorhersagemethoden (Modelle 1-3) auf das Training. Der Testzeitraum sei vom 01. April 2021 bis zum 20.Mai 2021. Innerhalb dieses Zeitraums werden drei Vorhersagen getroffen:
- Single-Period-Vorhersage (täglich eine neue Vorhersage für den nächsten Tag)
- Multi-Period-Vorhersage (ab 01. April 2021 für 7 Tage)
- Total-Period-Vorhersage (iterative Single-Period-Vorhersage für den gesamten Zeitraum)

Ziel ist es, mit den Vorhersagen die mittelere absolute Abweichung (Mean Absolute Error) zu minimieren. Zudem soll der R² Score berücksichtigt werden. Das Vorhersagemodell mit dem höchsten Potenzial soll in einer Webanwendung eingesetzt werden. 